<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#模型调优-(判定贷款用户是否逾期)" data-toc-modified-id="模型调优-(判定贷款用户是否逾期)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>模型调优 (判定贷款用户是否逾期)</a></span><ul class="toc-item"><li><span><a href="#数据集划分" data-toc-modified-id="数据集划分-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>数据集划分</a></span></li><li><span><a href="#模型评估" data-toc-modified-id="模型评估-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>模型评估</a></span></li><li><span><a href="#LR模型" data-toc-modified-id="LR模型-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>LR模型</a></span></li><li><span><a href="#SVM模型" data-toc-modified-id="SVM模型-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>SVM模型</a></span></li><li><span><a href="#决策树模型" data-toc-modified-id="决策树模型-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>决策树模型</a></span></li><li><span><a href="#XGBoost模型" data-toc-modified-id="XGBoost模型-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>XGBoost模型</a></span></li><li><span><a href="#LightGBM模型" data-toc-modified-id="LightGBM模型-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>LightGBM模型</a></span></li></ul></li><li><span><a href="#遇到的问题" data-toc-modified-id="遇到的问题-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>遇到的问题</a></span></li><li><span><a href="#Reference" data-toc-modified-id="Reference-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Reference</a></span></li></ul></div>

# 模型调优 (判定贷款用户是否逾期)

给定金融数据，预测贷款用户是否会逾期。
（status是标签：0表示未逾期，1表示逾期。）

**Task6（模型调优）** - 使用网格搜索对模型进行调优, 并采用五折交叉验证的方式进行模型评估

In [1]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

# 数据集预览
data = pd.read_csv('data.csv')
# 去重
data.drop_duplicates(inplace=True)
print(data.shape)
data.head()

(4754, 90)


,Unnamed: 0,custid,trade_no,bank_card_no,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,...,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
0,5,2791858,20180507115231274000000023057383,卡号1,0.01,0.99,0,0.90,0.55,0.313,...,2900.0,1688.0,1200.0,75.0,1.0,2.0,1200.0,1200.0,12.0,18.0
1,10,534047,20180507121002192000000023073000,卡号1,0.02,0.94,2000,1.28,1.00,0.458,...,3500.0,1758.0,15100.0,80.0,5.0,6.0,22800.0,9360.0,4.0,2.0
2,12,2849787,20180507125159718000000023114911,卡号1,0.04,0.96,0,1.00,1.00,0.114,...,1600.0,1250.0,4200.0,87.0,1.0,1.0,4200.0,4200.0,2.0,6.0
3,13,1809708,20180507121358683000000388283484,卡号1,0.00,0.96,2000,0.13,0.57,0.777,...,3200.0,1541.0,16300.0,80.0,5.0,5.0,30000.0,12180.0,2.0,4.0
4,14,2499829,20180507115448545000000388205844,卡号1,0.01,0.99,0,0.46,1.00,0.175,...,2300.0,1630.0,8300.0,79.0,2.0,2.0,8400.0,8250.0,22.0,120.0


In [2]:
import pickle
# 载入特征
with open('feature.pkl', 'rb') as f:
    X = pickle.load(f)

# 观测正负样本是否均衡
y = data.status
y.value_counts()

0    3561
1    1193
Name: status, dtype: int64

In [3]:
X.shape

(4754, 87)

## 数据集划分

In [4]:
# 划分训练集测试集
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=2333)

# 特征归一化
std = StandardScaler()
X_train = std.fit_transform(X_train)
X_test = std.transform(X_test)

## 模型评估

In [5]:
from sklearn.metrics import accuracy_score, roc_auc_score

def model_metrics(clf, X_train, X_test, y_train, y_test):
    # 预测
    y_train_pred = clf.predict(X_train)
    y_test_pred = clf.predict(X_test)
    
    y_train_proba = clf.predict_proba(X_train)[:,1]
    y_test_proba = clf.predict_proba(X_test)[:,1]
    
    # 准确率
    print('[准确率]', end = ' ')
    print('训练集：', '%.4f'%accuracy_score(y_train, y_train_pred), end = ' ')
    print('测试集：', '%.4f'%accuracy_score(y_test, y_test_pred))
    
    # auc取值：用roc_auc_score或auc
    print('[auc值]', end = ' ')
    print('训练集：', '%.4f'%roc_auc_score(y_train, y_train_proba), end = ' ')
    print('测试集：', '%.4f'%roc_auc_score(y_test, y_test_proba))

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from xgboost.sklearn import XGBClassifier
from lightgbm.sklearn import LGBMClassifier

## LR模型

In [7]:
lr = LogisticRegression()
param = {'C': [1e-3,0.01,0.1,1,10,100,1e3], 'penalty':['l1', 'l2']}

gsearch = GridSearchCV(lr, param_grid = param,scoring ='roc_auc', cv=5)
gsearch.fit(X_train, y_train)

print('最佳参数：',gsearch.best_params_)
print('训练集的最佳分数：', gsearch.best_score_)
print('测试集的最佳分数：', gsearch.score(X_test, y_test))

最佳参数： {'C': 0.1, 'penalty': 'l1'}
训练集的最佳分数： 0.7839975682166337
测试集的最佳分数： 0.7994893464372945


In [8]:
lr = LogisticRegression(C = 0.1, penalty = 'l1')
lr.fit(X_train, y_train)
model_metrics(lr, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.7959 测试集： 0.8073
[auc值] 训练集： 0.8016 测试集： 0.7995


## SVM模型

> 调参范围可设为'gamma':[0.001,0.01,0.1,1,10,100], 'C':[0.001,0.01,0.1,1,10,100]}。鉴于时间原因, 下面网格搜索时选用较小区间。

In [9]:
# 线性SVM
svm_linear = svm.SVC(kernel = 'linear', probability=True)
param = {'C':[0.01,0.1,1]}
gsearch = GridSearchCV(svm_linear, param_grid = param,scoring ='roc_auc', cv=5)
gsearch.fit(X_train, y_train)

print('最佳参数：',gsearch.best_params_)
print('训练集的最佳分数：', gsearch.best_score_)
print('测试集的最佳分数：', gsearch.score(X_test, y_test))

最佳参数： {'C': 0.01}
训练集的最佳分数： 0.7849963207712166
测试集的最佳分数： 0.810812878176418


In [10]:
svm_linear = svm.SVC(C = 0.01, kernel = 'linear', probability=True)
svm_linear.fit(X_train, y_train)
model_metrics(svm_linear, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.7848 测试集： 0.7912
[auc值] 训练集： 0.8044 测试集： 0.8108


In [11]:
# 多项式SVM
svm_poly = svm.SVC(kernel = 'poly', probability=True)
param = {'C':[0.01,0.1,1]}
gsearch = GridSearchCV(svm_poly, param_grid = param,scoring ='roc_auc', cv=5)
gsearch.fit(X_train, y_train)

print('最佳参数：',gsearch.best_params_)
print('训练集的最佳分数：', gsearch.best_score_)
print('测试集的最佳分数：', gsearch.score(X_test, y_test))

最佳参数： {'C': 0.01}
训练集的最佳分数： 0.7359832819347378
测试集的最佳分数： 0.7259879405573931


In [12]:
svm_poly =  svm.SVC(C = 0.01, kernel = 'poly', probability=True)
svm_poly.fit(X_train, y_train)
model_metrics(svm_poly, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.7538 测试集： 0.7547
[auc值] 训练集： 0.8690 测试集： 0.7260


In [13]:
# 高斯SVM
svm_rbf = svm.SVC(probability=True)
param = {'gamma':[0.01,0.1,1,10], 
         'C':[0.01,0.1,1]}
gsearch = GridSearchCV(svm_poly, param_grid = param,scoring ='roc_auc', cv=5)
gsearch.fit(X_train, y_train)

print('最佳参数：',gsearch.best_params_)
print('训练集的最佳分数：', gsearch.best_score_)
print('测试集的最佳分数：', gsearch.score(X_test, y_test))

最佳参数： {'C': 0.01, 'gamma': 0.01}
训练集的最佳分数： 0.7370458508629731
测试集的最佳分数： 0.7247746108112956


In [14]:
svm_rbf =  svm.SVC(gamma = 0.01, C =0.01 , probability=True)
svm_rbf.fit(X_train, y_train)
model_metrics(svm_rbf, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.7475 测试集： 0.7526
[auc值] 训练集： 0.8614 测试集： 0.7944


In [15]:
# sigmoid - SVM
svm_sigmoid = svm.SVC(kernel = 'sigmoid',probability=True)
param = {'C':[0.01,0.1,1]}
gsearch = GridSearchCV(svm_sigmoid, param_grid = param,scoring ='roc_auc', cv=5)
gsearch.fit(X_train, y_train)

print('最佳参数：',gsearch.best_params_)
print('训练集的最佳分数：', gsearch.best_score_)
print('测试集的最佳分数：', gsearch.score(X_test, y_test))

最佳参数： {'C': 0.01}
训练集的最佳分数： 0.7747312761692854
测试集的最佳分数： 0.7803266494690364


In [17]:
svm_sigmoid =  svm.SVC(C = 0.01, kernel = 'sigmoid',probability=True)
svm_sigmoid.fit(X_train, y_train)
model_metrics(svm_sigmoid, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.7475 测试集： 0.7526
[auc值] 训练集： 0.7615 测试集： 0.7803


## 决策树模型

1）首先对决策树最大深度max_depth和内部节点再划分所需最小样本数min_samples_split进行网格搜索。

In [18]:
param = {'max_depth':range(3,14,2), 'min_samples_split':range(100,801,200)}
gsearch = GridSearchCV(DecisionTreeClassifier(max_depth=8,min_samples_split=300,min_samples_leaf=20, max_features='sqrt',random_state =2333),
                       param_grid = param,scoring ='roc_auc', cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'max_depth': 11, 'min_samples_split': 100}, 0.7061428632294259)

2）对内部节点再划分所需最小样本数min_samples_split和叶子节点最少样本数min_samples_leaf一起调参。

In [19]:
param = {'min_samples_split':range(50,1000,100), 'min_samples_leaf':range(60,101,10)}
gsearch = GridSearchCV(DecisionTreeClassifier(max_depth=11,min_samples_split=100,min_samples_leaf=20, max_features='sqrt', random_state =2333),
                       param_grid = param,scoring ='roc_auc', cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'min_samples_leaf': 80, 'min_samples_split': 550}, 0.7118895755089746)

3）再对最大特征数max_features进行网格搜索

In [20]:
param = {'max_features':range(7,20,2)}
gsearch = GridSearchCV(DecisionTreeClassifier(max_depth=11,min_samples_split=550,min_samples_leaf=80, max_features='sqrt', random_state =2333),
                       param_grid = param,scoring ='roc_auc', cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'max_features': 19}, 0.7050922939313528)

In [21]:
dt = DecisionTreeClassifier(max_depth=11,min_samples_split=550,min_samples_leaf=80,max_features=19, random_state =2333)
dt.fit(X_train, y_train)
model_metrics(dt, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.7538 测试集： 0.7561
[auc值] 训练集： 0.7155 测试集： 0.6836


## XGBoost模型

1、max_depth = 5 :这个参数的取值最好在3-10之间。我选的起始值为5，但是你也可以选择其它的值。起始值在4-6之间都是不错的选择。

2、min_child_weight = 1:在这里选了一个比较小的值，因为这是一个极不平衡的分类问题。因此，某些叶子节点下的值会比较小。

3、gamma = 0: 起始值也可以选其它比较小的值，在0.1到0.2之间就可以。这个参数后继也是要调整的。

4、subsample, colsample_bytree = 0.8: 这个是最常见的初始值了。典型值的范围在0.5-0.9之间。

首先看一下默认参数的结果

In [22]:
import warnings
warnings.filterwarnings("ignore")

xgb0 = XGBClassifier()
xgb0.fit(X_train, y_train)

model_metrics(xgb0, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.8530 测试集： 0.8024
[auc值] 训练集： 0.9155 测试集： 0.7932


1) 首先从步长(learning rate)和迭代次数(n_estimators)入手。

开始选择一个较小的步长来网格搜索最好的迭代次数。这里，我们将步长初始值设置为0.1, 对于迭代次数进行网格搜索。

In [23]:
param_test = {'n_estimators':range(20,200,20)}
gsearch = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=140, max_depth=5, 
                                                  min_child_weight=1, gamma=0, subsample=0.8, 
                                                  colsample_bytree=0.8, objective= 'binary:logistic', 
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'n_estimators': 20}, 0.7822329511588559)

2) max_depth 和 min_child_weight 参数调优

In [24]:
param_test = {'max_depth':range(3,10,2), 'min_child_weight':range(1,12,2)}

gsearch = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=20, max_depth=5, 
                                                  min_child_weight=1, gamma=0, subsample=0.8, 
                                                  colsample_bytree=0.8, objective= 'binary:logistic', 
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'max_depth': 5, 'min_child_weight': 5}, 0.7852805853095239)

可以看出理想的max_depth值为5，理想的min_child_weight值为5。在这个值附近我们可以再进一步调整，来找出理想值。

In [25]:
param_test = {'max_depth':[3,4,5], 'min_child_weight':[3,4,5]}

gsearch = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=20, max_depth=5, 
                                                  min_child_weight=1, gamma=0, subsample=0.8, 
                                                  colsample_bytree=0.8, objective= 'binary:logistic', 
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'max_depth': 5, 'min_child_weight': 5}, 0.7852805853095239)

3) gamma参数调优

In [26]:
param_test = {'gamma':[i/10 for i in range(1,6)]}

gsearch = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=20, max_depth=5, 
                                                  min_child_weight=5, gamma=0, subsample=0.8, 
                                                  colsample_bytree=0.8, objective= 'binary:logistic', 
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'gamma': 0.4}, 0.7858965997168706)

4）调整subsample 和 colsample_bytree 参数

In [27]:
param_test = {'subsample':[i/10 for i in range(5,10)], 'colsample_bytree':[i/10 for i in range(5,10)]}

gsearch = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=20, max_depth=5, 
                                                  min_child_weight=5, gamma=0.4, subsample=0.8, 
                                                  colsample_bytree=0.8, objective= 'binary:logistic', 
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'colsample_bytree': 0.9, 'subsample': 0.9}, 0.7859006376180202)

从这里可以看出来，subsample理想取值0.9, colsample_bytree理想取值都是0.9。现在，我们以0.05为步长，在这个值附近尝试取值。

In [29]:
param_test = { 'subsample':[i/100 for i in range(85,101,5)], 'colsample_bytree':[i/100 for i in range(85,101,5)]}

gsearch = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=20, max_depth=5, 
                                                  min_child_weight=5, gamma=0.4, subsample=0.8, 
                                                  colsample_bytree=0.8, objective= 'binary:logistic', 
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'colsample_bytree': 0.95, 'subsample': 0.9}, 0.7868599289367876)

5）正则化参数调优

> #'reg_lambda': [0.2, 0.4, 0.6, 0.8, 1]

In [31]:
#  'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]
param_test = {'reg_alpha':[1e-5, 1e-2, 0.1, 0, 1, 100]}

gsearch = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=20, max_depth=5, 
                                                  min_child_weight=5, gamma=0.4, subsample=0.95, 
                                                  colsample_bytree=0.9, objective= 'binary:logistic', 
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'reg_alpha': 1}, 0.7855783149845719)

In [ ]:
6）继续从1）循环调整, 此处省略步骤。
最终选取的模型参数如下：

In [ ]:
#XGBClassifier(learning_rate =0.1, n_estimators=60, max_depth=3, min_child_weight=5, 
#              gamma=0.4, subsample=0.5, colsample_bytree=0.9, reg_alpha=1, objective= 'binary:logistic', 
#              nthread=4,scale_pos_weight=1, seed=27)             
# score -  0.7936928753627137

7）回到第1）步，降低学习速率, 调整迭代次数

In [46]:
param_test = {'n_estimators':range(20,200,20)}

gsearch = GridSearchCV(estimator = XGBClassifier(learning_rate =0.01, n_estimators=60, max_depth=3, 
                                                  min_child_weight=5, gamma=0.4, subsample=0.5, 
                                                  colsample_bytree=0.9, reg_alpha=1, objective= 'binary:logistic', 
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'n_estimators': 180}, 0.7842467214840007)

与1）结果相比, 较好。所以, 选用此处学习速率较小时的结果。继续从1）循环调整, 此处不再调整。

In [47]:
xgb = XGBClassifier(learning_rate =0.01, n_estimators=180, max_depth=3, min_child_weight=5, 
                    gamma=0.4, subsample=0.5, colsample_bytree=0.9, reg_alpha=1, 
                    objective= 'binary:logistic', nthread=4,scale_pos_weight=1, seed=27)
xgb.fit(X_train, y_train)
model_metrics(xgb, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.8028 测试集： 0.7940
[auc值] 训练集： 0.8257 测试集： 0.7953


## LightGBM模型

> 类似XGB， 先调整boosting框架参数, 再调整个体学习器参数。

In [48]:
# 首先看一下默认参数的结果
lgb0 = LGBMClassifier()
lgb0.fit(X_train, y_train)

model_metrics(lgb0, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.9955 测试集： 0.7849
[auc值] 训练集： 1.0000 测试集： 0.7804


In [59]:
param_test = {'n_estimators':range(20,200,20)}
# param_test = {'max_depth':range(3,10,2), 'min_child_weight':range(1,12,2)}
# param_test = {'gamma':[i/10 for i in range(1,6)]}
# param_test = {'subsample':[i/10 for i in range(5,10)], 'colsample_bytree':[i/10 for i in range(5,10)]}
# param_test = {'reg_alpha':[1e-5, 1e-2, 0.1, 0, 1, 100]}
gsearch = GridSearchCV(estimator = LGBMClassifier(learning_rate =0.1, n_estimators=60, max_depth=3, 
                                                  min_child_weight=11, gamma=0.1, subsample=0.5, 
                                                  colsample_bytree=0.8, reg_alpha = 1e-5,
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'n_estimators': 180}, 0.7853586425802856)

In [60]:
lgb = LGBMClassifier(learning_rate =0.1, n_estimators=60, max_depth=3, min_child_weight=11, 
                    gamma=0.1, subsample=0.5, colsample_bytree=0.8, reg_alpha=1e-5, 
                    nthread=4,scale_pos_weight=1, seed=27)
lgb.fit(X_train, y_train)
model_metrics(lgb, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.8242 测试集： 0.8066
[auc值] 训练集： 0.8715 测试集： 0.7970


|模型|参数|准确率|auc值|
|:---|:---|:---|:---|
|LR|LogisticRegression(C = 0.1, penalty = 'l1')|训练集： 0.7959 测试集： 0.8073|训练集： 0.8016 测试集： 0.7995|
|svm_linear|svm.SVC(C = 0.01, kernel = 'linear', probability=True)|训练集： 0.7848 测试集： 0.7912|训练集： 0.8044 测试集： 0.8108|
|svm_poly|svm.SVC(C = 0.01, kernel = 'poly', probability=True)|训练集： 0.7538 测试集： 0.7547| 训练集： 0.8690 测试集： 0.7260|
|svm_rbf|svm.SVC(gamma = 0.01, C =0.01 , probability=True)|训练集： 0.7475 测试集： 0.7526|训练集： 0.8614 测试集： 0.7944|
|svm_sigmoid|svm.SVC(C = 0.01, kernel = 'sigmoid',probability=True)|训练集： 0.7475 测试集： 0.7526|训练集： 0.7615 测试集： 0.7803|
|决策树|DecisionTreeClassifier(max_depth=11,min_samples_split=550,min_samples_leaf=80,max_features=19)|训练集： 0.7538 测试集： 0.7561|训练集： 0.7155 测试集： 0.6836|
|XGBoost|XGBClassifier(learning_rate =0.01, n_estimators=180, max_depth=3, min_child_weight=5,gamma=0.4, subsample=0.5, colsample_bytree=0.9, reg_alpha=1, objective= 'binary:logistic',nthread=4,scale_pos_weight=1, seed=27)|训练集： 0.8028 测试集： 0.7940|训练集： 0.8257 测试集： 0.7953|
|LightGBM|LGBMClassifier(learning_rate =0.1, n_estimators=60, max_depth=3, min_child_weight=11,gamma=0.1, subsample=0.5, colsample_bytree=0.8, reg_alpha=1e-5, nthread=4,scale_pos_weight=1, seed=27)|训练集： 0.8242 测试集： 0.8066|训练集： 0.8715 测试集： 0.7970|

# 遇到的问题

> XGB调参完还没有默认参数好, 一方面原因是因为没细调，另外是有5折时对score取平均的原因吗？

# Reference

[XGB调参](https://blog.csdn.net/han_xiaoyang/article/details/52665396)

[XGBoost和LightGBM的参数以及调参](https://www.jianshu.com/p/1100e333fcab)